# **Demography of Coders in 2020**

> Indented block


In this notebook, I will try to answer several question based on the data, such as


1.   What is the age trend of coders?
2.   
3.   



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

We load Pandas, Numpy, and pyplot for this project. This libraries will be heavily used for this project

**Load and Filter Data needed**

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Load data from csv
df = pd.read_csv('/content/drive/MyDrive/Dataset Udacity/Stack Overflow 2020/survey_results_public.csv')

# Show only the desired column
column = ['MainBranch','Age1stCode','Country','Hobbyist','YearsCode']

df = df[column]

df.head()

,MainBranch,Age1stCode,Country,Hobbyist,YearsCode
0,I am a developer by profession,13,Germany,Yes,36
1,I am a developer by profession,19,United Kingdom,No,7
2,I code primarily as a hobby,15,Russian Federation,Yes,4
3,I am a developer by profession,18,Albania,Yes,7
4,"I used to be a developer by profession, but no...",16,United States,Yes,15
